In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Read data from combined dataset
data = pd.read_csv('/content/drive/My Drive/combined_data.csv')

In [18]:
from sklearn.model_selection import train_test_split

#data.head(5)
# Sort by tiles
tile_groups = data[['Date', 'Reference']].drop_duplicates()

# Split groups — this keeps all 9 neighbors together
group_train, group_test = train_test_split(
    tile_groups, test_size=0.2, random_state=1234
)


# Join back to original data to get full rows
data_train = data.merge(group_train, on=['Date', 'Reference'], how='inner')
data_test = data.merge(group_test, on=['Date', 'Reference'], how='inner')

X = data.iloc[:, 5:10]
feature_columns = X.columns
feature_columns = list(feature_columns) + ['Date', 'Reference', 'Neighbor Number']

#print(X_train.head(5))

# Scale X data
scaler = MinMaxScaler()
data_train[X.columns] = scaler.fit_transform(data_train[X.columns])
data_test[X.columns] = scaler.transform(data_test[X.columns])

data_train.sort_values(by=['Reference','Neighbor Number', 'Date'], inplace=True)
data_test.sort_values(by=['Reference','Neighbor Number', 'Date'], inplace=True)

X_train = data_train[feature_columns]  # replace with your actual feature names
y_train = data_train['FROST']

X_test = data_test[feature_columns]
y_test = data_test['FROST']

X_train['Date'] = pd.to_datetime(X_train['Date'], format='%Y%m%d')
X_train['Date'] = X_train['Date'].apply(lambda x: x.toordinal())
X_train[X.columns] = scaler.transform(X_train[X.columns])

X_test['Date'] = pd.to_datetime(X_test['Date'], format='%Y%m%d')
X_test['Date'] = X_test['Date'].apply(lambda x: x.toordinal())
X_test[X.columns] = scaler.transform(X_test[X.columns])

<ipython-input-18-29a638af117c>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['Date'] = pd.to_datetime(X_train['Date'], format='%Y%m%d')
<ipython-input-18-29a638af117c>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['Date'] = X_train['Date'].apply(lambda x: x.toordinal())
<ipython-input-18-29a638af117c>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

In [20]:
X_train.head()

,WS2M,T2MDEW,ALLSKY_SFC_LW_DWN,ALLSKY_SFC_SW_DWN,T2M_MAX,Date,Reference,Neighbor Number
188440,0.000610,0.515830,-0.481657,-0.014430,0.315443,730120,Atlanta,1
188441,0.000166,0.516389,-0.480323,-0.013776,0.316368,730122,Atlanta,1
188442,0.000222,0.514254,-0.484312,-0.011665,0.315046,730123,Atlanta,1
188443,0.000222,0.510530,-0.491758,-0.006880,0.313754,730124,Atlanta,1
188444,0.001054,0.510986,-0.494284,-0.007019,0.313880,730125,Atlanta,1


In [38]:
# Grouping training data
reshaped_data = []
reshaped_y_train = []
sequence_length = 4

#make sure a group is only put together for a particular pixel
grouped = X_train.groupby(['Reference', 'Neighbor Number'])

# Loop through groups
for name, group in grouped:
    group = group.sort_index()
    num_sequences = len(group) // sequence_length

    for i in range(num_sequences):
        sequence = group[X.columns].iloc[i * sequence_length: (i + 1) * sequence_length].values
        reshaped_data.append(sequence)

        # Extract corresponding y label (last element of sequence)
        target_sequence = y_train[group.index[i * sequence_length: (i + 1) * sequence_length]].values
        reshaped_y_train.append(target_sequence[-1])

X_train_in = np.array(reshaped_data)
y_train_in = np.array(reshaped_y_train)

X_train_in shape: (241944, 4, 5)
y_train_in shape: (241944,)


In [39]:
X_train_tensor = torch.tensor(X_train_in, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train_in, dtype=torch.float32).to(device)

print(X_train_tensor.shape)
print(y_train_tensor.shape)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

torch.Size([241944, 4, 5])
torch.Size([241944])


In [42]:
# Grouping testing data
reshaped_test_data = []
reshaped_y_test = []
sequence_length = 4

grouped_test = X_test.groupby(['Reference', 'Neighbor Number'])

for name, group in grouped_test:
    group = group.sort_index()
    num_sequences = len(group) // sequence_length

    for i in range(num_sequences):
        sequence = group[X.columns].iloc[i * sequence_length: (i + 1) * sequence_length].values
        reshaped_test_data.append(sequence)

        target_sequence = y_test[group.index[i * sequence_length: (i + 1) * sequence_length]].values
        reshaped_y_test.append(target_sequence[-1])

X_test_in = np.array(reshaped_test_data)
y_test_in = np.array(reshaped_y_test)

X_test_tensor = torch.tensor(X_test_in, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test_in, dtype=torch.float32).to(device)

print(X_test_tensor.shape)
print(y_test_tensor.shape)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoader
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


torch.Size([30213, 4, 5])
torch.Size([30213])


In [41]:
# Feature size
dynamic_input_dim = X.shape[1]

class TemporalNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(TemporalNet, self).__init__()

        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=4, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x, _ = self.lstm(x)  # x shape: (batch, sequence_length, hidden_dim)
        x = self.fc(x[:, -1, :])  # Only use output from last timestep
        return x

# Initialize model
hidden_dim = 64
output_dim = 1 # Just predicting frost presence for each neighbor/pixel
model = TemporalNet(dynamic_input_dim, hidden_dim, output_dim).to(device)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss and Optimizer
pos_weight = torch.ones([1])*3
pos_weight = pos_weight.to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

optimizer = optim.Adam(model.parameters(), lr=0.001)
# Training Loop
num_epochs = 8
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        X_batch.to(device)
        y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        outputs = outputs.to(device)
        y_batch = y_batch.float()
        y_batch.to(device)
        # Reshape batch to get in same dimension as output
        y_batch = y_batch.view(-1, 1)
        y_batch = y_batch.to(device)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}')



Epoch 1/8, Loss: 0.5019
Epoch 2/8, Loss: 0.3990
Epoch 3/8, Loss: 0.3473
Epoch 4/8, Loss: 0.3325
Epoch 5/8, Loss: 0.3285
Epoch 6/8, Loss: 0.3274
Epoch 7/8, Loss: 0.3208
Epoch 8/8, Loss: 0.3137


In [52]:
# Evaluation
model.eval()
y_pred = []
y_true = []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        logits = model(X_batch)
        probs = torch.sigmoid(logits)
        y_pred.append(probs.cpu().numpy())
        y_batch = y_batch.float()
        y_batch = y_batch.view(-1, 1)
        y_true.append(y_batch.cpu().numpy())

y_pred = np.vstack(y_pred)
y_true = np.vstack(y_true)

predictions = []
probabilities = []
for prediction in y_pred:
    if prediction > 0.5:
        predictions.append(1)
    else:
        predictions.append(0)
    probabilities.append(prediction)

In [53]:
from sklearn import metrics

accuracy = metrics.accuracy_score(list(y_true), predictions)
print(f'accuracy: {accuracy}')

precision = metrics.precision_score(list(y_true), predictions)
print(f'precision: {precision}')

recall = metrics.recall_score(list(y_true), predictions)
print(f'recall: {recall}')

f1 = metrics.f1_score(list(y_true), predictions)
print(f'f1 score: {f1}')

accuracy: 0.8724059179823255
precision: 0.41508152173913043
recall: 0.8560420315236428
f1 score: 0.5590758320942468


In [49]:
from sklearn import metrics
auc = metrics.roc_auc_score(list(y_true), predictions)
print(f'auc score: {auc}')

tn, fp, fn, tp = metrics.confusion_matrix(list(y_true), predictions).ravel()

print(metrics.confusion_matrix(list(y_true), predictions))

auc score: 0.8650778181596575
[[23914  3444]
 [  411  2444]]
